In [39]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['MCA_WT']

# Define and insert data for the 'policies' collection
policies = db['policies']
policies.delete_many({})
policies_data = [
    {"_id": 1, "policy_name": "Komal Jeevan", "policy_type": "Half Yearly", "premium_amount": 5000, "company": "Company A", "customers" : [1,2]},
    {"_id": 2, "policy_name": "Life Secure", "policy_type": "Half Yearly", "premium_amount": 12000, "company": "Company B", "customers" : [1,4]},
    {"_id": 3, "policy_name": "Health Guard", "policy_type": "Half Yearly", "premium_amount": 7000, "company": "Company A", "customers" : [2]},
    {"_id": 4, "policy_name": "Family Protect", "policy_type": "Monthly", "premium_amount": 4000, "company": "Company D", "customers" : [3,5]}
]
policies.insert_many(policies_data)

# Define and insert data for the 'customers' collection
customers = db['customers']
customers.delete_many({})
customers_data = [
    {"_id": 1, "name": "Alice", "age": 30, "policies": [1, 2]},
    {"_id": 2, "name": "Bob", "age": 45, "policies": [1, 3]},
    {"_id": 3, "name": "Charlie", "age": 29, "policies": [4]},
    {"_id": 4, "name": "David", "age": 35, "policies": [2]},
    {"_id": 5, "name": "Eve", "age": 40, "policies": [4]}
]
customers.insert_many(customers_data)


InsertManyResult([1, 2, 3, 4, 5], acknowledged=True)

In [40]:
# List the details of customers who have taken “Komal Jeevan” Policy

def fun(policy_name):
    ids = policies.find_one({'policy_name':policy_name},{"customers":1,"_id":0})
    cid = ids['customers']
    if cid:
        names = customers.find({"_id": {"$in": cid}},{"name":1,"_id":0})
        return [ n['name'] for n in names]
    return []

# Example usage:
lst = fun("Komal Jeevan")
print("Customers with 'Komal Jeevan' Policy:", lst)

Customers with 'Komal Jeevan' Policy: ['Alice', 'Bob']


In [41]:
# Display average premium amount

def average_premium_amount():
    # groups all documents together (since _id is set to None) and calculates the average value of the premium_amount field across those documents, storing the result in a new field called average_premium.
    pipeline = [
        {"$group": {"_id": None, "average_premium": {"$avg": "$premium_amount"}}}  #It calculates the average of the premium_amount field across all documents.
    ]
    result = policies.aggregate(pipeline)
    for i in result:
        avg = i['average_premium']
    return avg

# Example usage:
avg_premium = average_premium_amount()
print("Average Premium Amount:", avg_premium)

Average Premium Amount: 7000.0


In [42]:
#  Increase the premium amount by 5% for policy type “Monthly”
def increase_monthly_premium():
    policies.update_many(
        {"policy_type": "Monthly"},
        {"$mul": {"premium_amount": 0.05}}
        # $mul operator in MongoDB is used to perform multiplication on a field's value
    )

# Example usage:
increase_monthly_premium()
print("Increased premium amounts for monthly policies.")

Increased premium amounts for monthly policies.


In [45]:
# Count the number of customers who have taken policy type “Half Yearly”

def count_customers_with_half_yearly_policy():
    policy = policies.find({"policy_type": "Half Yearly"}, {"_id": 0,'customers':1})
    cid = []
    for p in policy:
        cid.extend(p['customers'])
    cid = list(set(cid))

    cnames = customers.find({"_id":{"$in":cid}},{'name':1,"_id":0})
    if cnames:
        return [ c['name'] for c in cnames]
    return []
# Example usage:
half_yearly_customer_count = count_customers_with_half_yearly_policy()
print("Number of customers with 'Half Yearly' policy:", half_yearly_customer_count)

Number of customers with 'Half Yearly' policy: ['Alice', 'Bob', 'David']
